In [1]:
import datasets
import tempfile
import logging
import random
import config  # 这里的config是官方自带的  我们导入了
import os
import yaml
import logging
import time
import torch
import transformers
# import os
# os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

from utilities import * # 这里的utilities是官方自带的  我们导入了
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
import lamini
from lamini import Lamini
lamini.api_key = "95b701553ffa985833e95cff20d4606ca194adb5614624ce9e9a49ec4e980929"

# from llama import BasicModelRunner
# from llama import BasicModelRunner

logger = logging.getLogger(__name__)
global_config = None

/Applications/anaconda3/envs/llm-universe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-16 23:02:10.431993: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 数据导入

In [3]:
dataset_path = "data/lamini_docs.jsonl"
use_hf = False

In [4]:
model_name = "EleutherAI/pythia-70m"
my_training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048  # 最长输入
    },
    "datasets": {  # 数据设置
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token#end of sequences填充

print("Training Config: ", my_training_config)  # Debugging
train_dataset, test_dataset = tokenize_and_split_data(training_config=my_training_config, tokenizer=tokenizer)
print("Training Dataset: ", train_dataset)  # Debugging
print("Test Dataset: ", test_dataset)  # Debugging

/Applications/anaconda3/envs/llm-universe/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2024-12-16 22:05:15,912 - DEBUG - utilities - Config: datasets.path: data/lamini_docs.jsonl
datasets.use_hf: false
model.max_length: 2048
model.pretrained_name: EleutherAI/pythia-70m
verbose: true



Training Config:  {'model': {'pretrained_name': 'EleutherAI/pythia-70m', 'max_length': 2048}, 'datasets': {'use_hf': False, 'path': 'data/lamini_docs.jsonl'}, 'verbose': True}
def:training_config {'model': {'pretrained_name': 'EleutherAI/pythia-70m', 'max_length': 2048}, 'datasets': {'use_hf': False, 'path': 'data/lamini_docs.jsonl'}, 'verbose': True}
tokenize False data/lamini_docs.jsonl


2024-12-16 22:05:18,348 - DEBUG - fsspec.local - open file: /Users/kangxun/.cache/huggingface/datasets/json/default-c0432dc788164e4e/0.0.0/f4e89e8750d5d5ffbef2c078bf0ddfedef29dc2faff52a6255cf513c05eb1092/dataset_info.json
2024-12-16 22:05:18,381 - DEBUG - fsspec.local - open file: /Users/kangxun/.cache/huggingface/datasets/json/default-c0432dc788164e4e/0.0.0/f4e89e8750d5d5ffbef2c078bf0ddfedef29dc2faff52a6255cf513c05eb1092/dataset_info.json


Training Dataset:  Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1260
})
Test Dataset:  Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 140
})


### 加载基础模型

In [6]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

device_count = torch.cuda.device_count()

if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

base_model.to(device)

2024-12-16 22:05:19,828 - DEBUG - __main__ - Select CPU device


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [7]:
def inference(text, model, tokenizer, max_input_tokens=1000, max_output_token=100):
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_tokens
    )
    #Generate 模型生成
    device = model.device
    generated_tokens_with_prompt = model.generate(
        input_ids = input_ids.to(device),
        max_length=max_output_token
    )

    generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)
     # Strip the prompt 切除prompt
    generated_text_answer = generated_text_with_prompt[0][len(text):]
    return generated_text_answer

### 使用基础模型

In [8]:
test_text = test_dataset[0]['question']
print("Question Input (test):", test_text)
print(f"Correct answer from Lamini docs: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question Input (test): Can Lamini generate technical documentation or user manuals for software projects?
Correct answer from Lamini docs: Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.
Model's answer: 


I have a question about the following:

How do I get the correct documentation to work?

A:

I think you need to use the following code:

A:

You can use the following code to get the correct documentation.

A:

You can use the following code to get the correct documentation.

A:

You can use the following


### 训练参数配置

In [9]:
max_steps = 3

trained_model_name = f"lamini_docs_{max_steps}_steps"
output_dir = trained_model_name

training_args = TrainingArguments(
    learning_rate = 1.0e-5,
    num_train_epochs=1,
    max_steps=max_steps,
    per_device_train_batch_size=1,
    # dispatch_batches=True,
    output_dir=output_dir,

     # 其它参数
    overwrite_output_dir=False, # 覆盖输出文件夹
    disable_tqdm=False, # 禁用进度条
    eval_steps=120, # 评估模型的步数间隔
    save_steps=120, # 模型存储步数间隔
    warmup_steps=1, # 学习率调度器的预热步数
    per_device_eval_batch_size=1, # 评估的批大小
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=1,
    optim="adafactor",
    gradient_accumulation_steps = 4,
    gradient_checkpointing=False,

    # 提前停止参数
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)



In [10]:
# print(training_args.dispatch_batches is None)
# print(training_args.deepspeed_plugin is None)

In [11]:
#模型执行的浮点运算 (FLOPs)的数量
#模型计算复杂度和资源消耗情况
model_flops = (
    base_model.floating_point_ops(
        {
            "input_ids":torch.zeros(
                (1, my_training_config["model"]["max_length"])
            )
        }
    )
    * training_args.gradient_accumulation_steps
)
print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

### 微调开始训练

In [13]:
# from accelerate import Accelerator, DataLoaderConfiguration

# # Define DataLoaderConfiguration
# dataloader_config = DataLoaderConfiguration(
#     dispatch_batches=False,  # Each process fetches its own batch
#     split_batches=True       # Split fetched batches across processes
# )

# # Initialize Accelerator with DataLoaderConfiguration
# accelerator = Accelerator(dataloader_config=dataloader_config)

# 训练器装载
# accelerate==0.28.0 Accelerator受dispatch_batches参数影响，老版本不用传值，新版本必须

trainer = Trainer(
    model=base_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [15]:
training_output = trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:01<00:02,  1.13s/it]

{'loss': 3.3405, 'learning_rate': 1e-05, 'epoch': 0.0, 'iter_time': 0.0, 'flops': 0.0, 'remaining_time': 0.0}


 67%|██████▋   | 2/3 [00:02<00:01,  1.03s/it]

{'loss': 3.2429, 'learning_rate': 5e-06, 'epoch': 0.01, 'iter_time': 0.9609012603759766, 'flops': 2285008775504.041, 'remaining_time': 0.9609012603759766}


100%|██████████| 3/3 [00:03<00:00,  1.05s/it]

{'loss': 3.4015, 'learning_rate': 0.0, 'epoch': 0.01, 'iter_time': 1.0090715885162354, 'flops': 2175928682701.8547, 'remaining_time': 0.0}
{'train_runtime': 3.1585, 'train_samples_per_second': 3.799, 'train_steps_per_second': 0.95, 'train_loss': 3.328324476877848, 'epoch': 0.01, 'iter_time': 1.011546015739441, 'flops': 2170605961753.4702, 'remaining_time': 0.0}


In [16]:
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

Saved model to: lamini_docs_3_steps/final


In [17]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)

In [18]:
finetuned_slightly_model.to(device)

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (a

In [19]:
test_question = test_dataset[0]['question']
print("Question input (test):", test_question)

print("Finetuned slightly model's answer: ")
print(inference(test_question, finetuned_slightly_model, tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question input (test): Can Lamini generate technical documentation or user manuals for software projects?
Finetuned slightly model's answer: 


I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamini-specific software development process. I have a question about the Lamin


In [20]:
test_answer = test_dataset[0]['answer']
print("Target answer output (test):", test_answer)

Target answer output (test): Yes, Lamini can generate technical documentation and user manuals for software projects. It uses natural language generation techniques to create clear and concise documentation that is easy to understand for both technical and non-technical users. This can save developers a significant amount of time and effort in creating documentation, allowing them to focus on other aspects of their projects.


In [21]:
# 运行对比一下训练过的在线模型（这个模型是官方微调过得，轮数更多，调整更多次参数的模型。）
finetuned_longer_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")
tokenizer = AutoTokenizer.from_pretrained("lamini/lamini_docs_finetuned")

finetuned_longer_model.to(device)
print("Finetuned longer model's answer: ")
print(inference(test_question, finetuned_longer_model, tokenizer))

/Applications/anaconda3/envs/llm-universe/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Finetuned longer model's answer: 
Yes, Lamini can generate technical documentation or user manuals for software projects. This can be achieved by providing a prompt for a specific technical question or question to the LLM Engine, or by providing a prompt for a specific technical question or question. Additionally, Lamini can be trained on specific technical questions or questions to help users understand the process and provide feedback to the LLM Engine. Additionally, Lamini


In [22]:
### 更小微调模型
base_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
base_model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")
print(inference("What do you think of Mars?", base_model, base_tokenizer))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.




I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I think I’m going to go to the next page.

I


In [23]:
finetuned_longer_model = AutoModelForCausalLM.from_pretrained("lamini/lamini_docs_finetuned")
tokenizer = AutoTokenizer.from_pretrained("lamini/lamini_docs_finetuned")
print("What do you think of Mars?",inference("What do you think of Mars?", finetuned_longer_model, tokenizer))
# 可以看到话题拉回来了  微调成功~

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


What do you think of Mars? Let’s keep the discussion relevant to Lamini. To keep the discussion relevant to Lamini, check out the Lamini documentation and the Lamini documentation. For more information, visit https://lamini-ai.github.io/Lamini/. For more information, visit https://lamini-ai.github.io/. For more information, visit https://lamini-ai.github.io/. For more


In [16]:
llm = Lamini("EleutherAI/pythia-70m")

# data = [
#     {
#         "input": "<|begin_of_text|><|start_header_id|>user<|end_header_id|>Which animal has the best memory?<|eot_id|><|start_header_id|>assistant<|end_header_id|>",
#         "output": "A llama!<|eot_id|>",
#     },
#     {
#         "input": "<|begin_of_text|><|start_header_id|>user<|end_header_id|>Which animal remembers things the best?<|eot_id|><|start_header_id|>assistant<|end_header_id|>",
#         "output": "A llama!<|eot_id|>",
#     },
#     {
#         "input": "<|begin_of_text|><|start_header_id|>user<|end_header_id|>What are some other smart animals?<|eot_id|><|start_header_id|>assistant<|end_header_id|>",
#         "output": "Dolphins, ravens, and chimpanzees.<|eot_id|>",
#     },
# ]

# llm.tune(data_or_dataset_id=data)

AttributeError: 'Lamini' object has no attribute 'layers'

In [12]:
from lamini import Lamini

llm = Lamini("EleutherAI/pythia-410m")
data = [
    {
        "question": "<|begin_of_text|><|start_header_id|>user<|end_header_id|>What are the different types of documents available in the repository (e.g., installation guide, API documentation, developer's guide)?<|eot_id|><|start_header_id|>assistant<|end_header_id|>",
        "answer": "Lamini has documentation on Getting Started, Authentication, Question Answer Model, Python Library, Batching, Error Handling, Advanced topics, and class documentation on LLM Engine available at https://lamini-ai.github.io/.<|eot_id|>",
    },
    {
        "question": "<|begin_of_text|><|start_header_id|>user<|end_header_id|>What is the recommended way to set up and configure the code repository?<|eot_id|><|start_header_id|>assistant<|end_header_id|>",
        "answer": "Lamini can be downloaded as a python package and used in any codebase that uses python. Additionally, we provide a language agnostic REST API. We’ve seen users develop and train models in a notebook environment, and then switch over to a REST API to integrate with their production environment.<|eot_id|>",
    },
]

llm.tune(data_or_dataset_id=data)

Data pairs uploaded to local.

Your dataset id is: 6278999797d0b90f686801d04af106a811f5e45f1337eed30447b7f6da616ec5 . Consider using this in the future to train using the same data. 
Eg: llm.train(data_or_dataset_id='6278999797d0b90f686801d04af106a811f5e45f1337eed30447b7f6da616ec5')
Tuning job submitted! Check status of job 14823 here: https://api.lamini.ai/train/14823


{'job_id': 14823,
 'status': 'CREATED',
 'dataset_id': '6278999797d0b90f686801d04af106a811f5e45f1337eed30447b7f6da616ec5'}

In [8]:
# 上面这个错误大家应该都会遇到，因为EleutherAI/pythia-410m需要会员，我们用相近的pythia-410m-v0替代。
from lamini import Lamini

# Initialize the model
model = Lamini(model_name="EleutherAI/pythia-70m")

# Prepare data
data = [
    {   
        "question": "What are the different types of documents available in the repository (e.g., installation guide, API documentation, developer's guide)?", 
        "answer": "Lamini has documentation on Getting Started, Authentication, Question Answer Model, Python Library, Batching, Error Handling, Advanced topics, and class documentation on LLM Engine available at https://lamini-ai.github.io/."
    },
    {
        "question": "What is the recommended way to set up and configure the code repository?", 
        "answer": "Lamini can be downloaded as a python package and used in any codebase that uses python. Additionally, we provide a language agnostic REST API. We’ve seen users develop and train models in a notebook environment, and then switch over to a REST API to integrate with their production environment."
    },
]

# Tune the model
try:
    model.tune(data_or_dataset_id=data)
    print("Model tuning completed successfully.")
except Exception as e:
    print(f"Error during model tuning: {e}")
# model.load_data_from_jsonlines("data/lamini_docs.jsonl")

# dataset_id = model.upload_file("data/lamini_docs.jsonl", input_key="question", output_key="answer")
# model.train(dataset_id)
# model.tune(data_or_dataset_id=dataset_id) # -> returns an ID, dashboard, and chat interface

Data pairs uploaded to local.

Your dataset id is: 90f3cfa9b43a2b4e3baccecb09a2add064b65cb5dab293f0cfca1d8af56f8fb7 . Consider using this in the future to train using the same data. 
Eg: llm.train(data_or_dataset_id='90f3cfa9b43a2b4e3baccecb09a2add064b65cb5dab293f0cfca1d8af56f8fb7')
Tuning job submitted! Check status of job 14820 here: https://api.lamini.ai/train/14820
Model tuning completed successfully.


In [35]:
save_dir = f'{output_dir}/final_3lines'

model.save_model(save_dir)
print("Saved model to:", save_dir)

AttributeError: 'Lamini' object has no attribute 'save_model'

In [39]:
import os
print(os.path.exists("data/lamini_docs.jsonl"))  

True


In [17]:
llm = Lamini(model_name='meta-llama/Meta-Llama-3.1-8B-Instruct')

dataset_id = llm.upload_file("datasets/devops_docs.jsonl", input_key="question", output_key="answer")
llm.train(data_or_dataset_id=dataset_id)

Error uploading data pairs: [Errno 2] No such file or directory: 'datasets/devops_docs.jsonl'
Error reading data file: [Errno 2] No such file or directory: 'datasets/devops_docs.jsonl'


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/devops_docs.jsonl'